In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy import sparse
from sklearn.linear_model import Ridge,RidgeCV

In [2]:
train = pd.read_csv('howpop_train.csv')
test = pd.read_csv('howpop_test.csv')

In [3]:
train. shape , test.shape

((134137, 17), (3990, 9))

In [4]:
train

,url,domain,post_id,published,author,flow,polling,content_len,title,comments,favs,views,votes_plus,votes_minus,views_lognorm,favs_lognorm,comments_lognorm
0,https://habrahabr.ru/post/18284/,habrahabr.ru,18284,2008-01-01 18:19:00,@Tapac,develop,False,4305,Новогодний подарок блоггерам — WordPress 2.3.2,0,0,236,0.0,0.0,-0.792687,-1.344075,-2.436871
1,https://habrahabr.ru/post/18285/,habrahabr.ru,18285,2008-01-01 18:30:00,@DezmASter,design,False,7344,"Сумасшедшие яйца, или сервис для отслеживания ...",1,1,353,1.0,0.0,-0.485214,-0.831946,-1.873193
2,https://habrahabr.ru/post/18286/,habrahabr.ru,18286,2008-01-01 18:34:00,@DezmASter,design,False,8431,"Сумасшедшие яйца, или сервис для отслеживания ...",47,72,1200,45.0,5.0,0.446090,1.794023,0.574656
3,https://habrahabr.ru/post/18291/,habrahabr.ru,18291,2008-01-02 01:32:00,@Taoorus,design,False,5662,"Сглаживание шрифтов, и субпиксельная отрисовка",102,36,5700,72.0,3.0,1.627469,1.300050,1.167375
4,https://geektimes.ru/post/18294/,geektimes.ru,18294,2008-01-02 14:34:00,@dennydo,NaN,False,3706,Почему мне не нравится iPhone,230,6,1400,73.0,15.0,1.362397,0.961094,2.020356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134132,https://habrahabr.ru/company/mailru/blog/314014/,habrahabr.ru,314014,2016-10-31 21:45:00,NaN,develop,False,56033,Парсинг JSON — это минное поле,58,347,35500,164.0,6.0,1.870174,2.124862,1.258447
134133,https://geektimes.ru/post/282086/,geektimes.ru,282086,2016-10-31 22:44:00,@marks,NaN,False,5504,Заработала система раннего предупреждения НАСА...,52,16,12900,22.0,0.0,-0.091464,-0.511814,0.465118
134134,https://geektimes.ru/post/282090/,geektimes.ru,282090,2016-10-31 23:26:00,@Shapelez,NaN,False,2883,Арзамасский городской суд постановил заблокиро...,16,14,15800,28.0,4.0,0.198924,-0.656390,-0.408714
134135,https://geektimes.ru/post/282092/,geektimes.ru,282092,2016-10-31 23:52:00,@ragequit,NaN,False,5735,Медиа-коммуникационный союз планирует внести в...,47,14,15000,20.0,0.0,0.124244,-0.655385,0.389636


In [5]:
test

,url,domain,post_id,published,author,flow,polling,content_len,title
0,https://habrahabr.ru/post/314080/,habrahabr.ru,314080,2016-11-01 01:05:00,@fsou11,develop,True,20132,Опыт использования MassTransit 3.0
1,https://habrahabr.ru/company/plesk/blog/313732/,habrahabr.ru,313732,2016-11-01 05:42:00,NaN,marketing,False,12389,Геймификация форума на движке XenForo
2,https://habrahabr.ru/company/etagi/blog/314000/,habrahabr.ru,314000,2016-11-01 08:33:00,NaN,admin,False,72839,Кластер высокой доступности на postgresql 9.6 ...
3,https://habrahabr.ru/company/knopka/blog/314030/,habrahabr.ru,314030,2016-11-01 09:00:00,NaN,develop,False,29572,Как перестать бояться и полюбить синтаксически...
4,https://geektimes.ru/company/audiomania/blog/2...,geektimes.ru,282058,2016-11-01 09:44:00,NaN,NaN,False,9642,Мифы и реальность: Что нужно знать о современн...
...,...,...,...,...,...,...,...,...,...
3985,https://geektimes.ru/post/281642/,geektimes.ru,281642,2017-01-26 09:40:00,@PatientZero,NaN,False,23613,История разработки Deus Ex
3986,https://habrahabr.ru/company/tuturu/blog/320326/,habrahabr.ru,320326,2017-01-26 09:43:00,NaN,develop,False,19504,"Все лучшее из методологии Lean Startup, и как ..."
3987,https://habrahabr.ru/company/gemaltorussia/blo...,habrahabr.ru,320214,2017-01-26 10:00:00,NaN,develop,False,8395,Защита больших данных – как правильно начать и...
3988,https://habrahabr.ru/post/320472/,habrahabr.ru,320472,2017-01-26 10:01:00,@Reason89,develop,False,7788,Подключение Xmega к FRAM по SPI


In [6]:
set(train.columns)-set(test.columns) , set(test.columns)-set(train.columns)

({'comments',
  'comments_lognorm',
  'favs',
  'favs_lognorm',
  'views',
  'views_lognorm',
  'votes_minus',
  'votes_plus'},
 set())

In [7]:
feature = ['comments','comments_lognorm','favs','favs_lognorm','views','views_lognorm','votes_minus','votes_plus']
X , y = train.drop(feature,axis=1) , train.favs_lognorm

In [8]:
X.shape , y.shape

((134137, 9), (134137,))

In [9]:
X.drop('url',axis=1,inplace=True) 

In [10]:
X.isna().sum()

domain             0
post_id            0
published          0
author         36480
flow           37089
polling            0
content_len        0
title              0
dtype: int64

In [11]:
X['author'] = X['author'].fillna('-')
X['flow'] = X['flow'].fillna('-')

In [12]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134137 entries, 0 to 134136
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   domain       134137 non-null  object
 1   post_id      134137 non-null  int64 
 2   published    134137 non-null  object
 3   author       134137 non-null  object
 4   flow         134137 non-null  object
 5   polling      134137 non-null  bool  
 6   content_len  134137 non-null  int64 
 7   title        134137 non-null  object
dtypes: bool(1), int64(2), object(5)
memory usage: 7.3+ MB


In [13]:
X

,domain,post_id,published,author,flow,polling,content_len,title
0,habrahabr.ru,18284,2008-01-01 18:19:00,@Tapac,develop,False,4305,Новогодний подарок блоггерам — WordPress 2.3.2
1,habrahabr.ru,18285,2008-01-01 18:30:00,@DezmASter,design,False,7344,"Сумасшедшие яйца, или сервис для отслеживания ..."
2,habrahabr.ru,18286,2008-01-01 18:34:00,@DezmASter,design,False,8431,"Сумасшедшие яйца, или сервис для отслеживания ..."
3,habrahabr.ru,18291,2008-01-02 01:32:00,@Taoorus,design,False,5662,"Сглаживание шрифтов, и субпиксельная отрисовка"
4,geektimes.ru,18294,2008-01-02 14:34:00,@dennydo,-,False,3706,Почему мне не нравится iPhone
...,...,...,...,...,...,...,...,...
134132,habrahabr.ru,314014,2016-10-31 21:45:00,-,develop,False,56033,Парсинг JSON — это минное поле
134133,geektimes.ru,282086,2016-10-31 22:44:00,@marks,-,False,5504,Заработала система раннего предупреждения НАСА...
134134,geektimes.ru,282090,2016-10-31 23:26:00,@Shapelez,-,False,2883,Арзамасский городской суд постановил заблокиро...
134135,geektimes.ru,282092,2016-10-31 23:52:00,@ragequit,-,False,5735,Медиа-коммуникационный союз планирует внести в...


In [14]:
X = pd.concat([X, pd.get_dummies(X['flow'],prefix='flow')],axis=1)
X = pd.concat([X, pd.get_dummies(X['polling'],prefix='polling')],axis=1)
X = pd.concat([X, pd.get_dummies(X['domain'],prefix='domain')],axis=1)

In [15]:
X.drop(['flow','flow_-','domain','polling'],axis=1,inplace=True)

In [16]:
X['author'].value_counts()

-                36480
@alizar           5292
@marks            4711
@SLY_G            1939
@ivansychev       1350
                 ...  
@ph0enix             1
@fronik              1
@vizgl               1
@Co6aka              1
@Roman_Pekhov        1
Name: author, Length: 22078, dtype: int64

In [17]:
X['published'] = pd.to_datetime(X['published'],yearfirst=True)
X['year'] = [d.year for d in X.published]
X['month'] = [d.month for d in X.published]
X['dayofweek'] = [d.isoweekday() for d in X.published]
X['hour'] = [d.hour for d in X.published]

In [18]:
X.drop('published',axis=1,inplace=True)

In [19]:
X

,post_id,author,content_len,title,flow_admin,flow_design,flow_develop,flow_management,flow_marketing,flow_misc,polling_False,polling_True,domain_geektimes.ru,domain_habrahabr.ru,year,month,dayofweek,hour
0,18284,@Tapac,4305,Новогодний подарок блоггерам — WordPress 2.3.2,0,0,1,0,0,0,1,0,0,1,2008,1,2,18
1,18285,@DezmASter,7344,"Сумасшедшие яйца, или сервис для отслеживания ...",0,1,0,0,0,0,1,0,0,1,2008,1,2,18
2,18286,@DezmASter,8431,"Сумасшедшие яйца, или сервис для отслеживания ...",0,1,0,0,0,0,1,0,0,1,2008,1,2,18
3,18291,@Taoorus,5662,"Сглаживание шрифтов, и субпиксельная отрисовка",0,1,0,0,0,0,1,0,0,1,2008,1,3,1
4,18294,@dennydo,3706,Почему мне не нравится iPhone,0,0,0,0,0,0,1,0,1,0,2008,1,3,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134132,314014,-,56033,Парсинг JSON — это минное поле,0,0,1,0,0,0,1,0,0,1,2016,10,1,21
134133,282086,@marks,5504,Заработала система раннего предупреждения НАСА...,0,0,0,0,0,0,1,0,1,0,2016,10,1,22
134134,282090,@Shapelez,2883,Арзамасский городской суд постановил заблокиро...,0,0,0,0,0,0,1,0,1,0,2016,10,1,23
134135,282092,@ragequit,5735,Медиа-коммуникационный союз планирует внести в...,0,0,0,0,0,0,1,0,1,0,2016,10,1,23


In [20]:
%%time
tfid_vec = TfidfVectorizer(min_df=5,ngram_range=(1,2))
X_title_sparse = tfid_vec.fit_transform(X['title'])
X_author_space = tfid_vec.fit_transform(X['author'])

Wall time: 5.98 s


In [21]:
X_author_space , X_title_sparse

(<134137x3944 sparse matrix of type '<class 'numpy.float64'>'
 	with 67529 stored elements in Compressed Sparse Row format>,
 <134137x35071 sparse matrix of type '<class 'numpy.float64'>'
 	with 916344 stored elements in Compressed Sparse Row format>)

In [22]:
X.drop(['title','author'],axis=1,inplace=True)

In [23]:
X_sparse = sparse.csr_matrix(X)
X_sparse

<134137x16 sparse matrix of type '<class 'numpy.int64'>'
	with 1166065 stored elements in Compressed Sparse Row format>

In [24]:
X_sparse = sparse.hstack([X_sparse,X_title_sparse,X_author_space])
X_sparse

<134137x39031 sparse matrix of type '<class 'numpy.float64'>'
	with 2149938 stored elements in COOrdinate format>

In [25]:
X_train_sparse , X_valid_sparse , y_train , y_valid = train_test_split(X_sparse,y,test_size=0.3,random_state=17)
X_train_sparse.shape , X_valid_sparse.shape , y_train.shape , y_valid.shape 

((93895, 39031), (40242, 39031), (93895,), (40242,))

In [26]:
%%time
ridge = Ridge(alpha=0.1,random_state=17)
ridge.fit(X_train_sparse,y_train)
mean_squared_error(y_valid,ridge.predict(X_valid_sparse))

Wall time: 138 ms


0.865530210790848

In [27]:
%%time
param_grid = {'alpha':[i/10 for i in range(1,11)]}
grid_search_ridge = GridSearchCV(estimator=Ridge(random_state=17),param_grid=param_grid,cv=10,n_jobs=-1)
grid_search_ridge.fit(X_train_sparse,y_train)
mean_squared_error(y_valid,grid_search_ridge.predict(X_valid_sparse)) , grid_search_ridge.best_estimator_

Wall time: 8.42 s


(0.8655302107908472, Ridge(random_state=17))

In [28]:
%%time
ridge_cv = RidgeCV(cv=5)
ridge_cv.fit(X_train_sparse,y_train)
mean_squared_error(y_valid,ridge_cv.predict(X_valid_sparse))

Wall time: 1.77 s


0.8655302107908417